In [ ]:
class Queue():
    '''
    FIFO Queue
    '''
    def __init__(self, input_list=[]):
        self.list = input_list
        
    def push(self, el):
        self.list.append(el)
        
    def pop(self):
        a = self.list.pop(0)
        return a
    
    def is_empty(self):
        return len(self.list) == 0

In [3]:
class Stack():
    '''
    LIFO Stack
    '''
    def __init__(self, input_list=[]):
        self.list = input_list
        
    def push(self, el):
        self.list.append(el)
        
    def pop(self):
        a = self.list.pop()
        return a
    
    def is_empty(self):
        return len(self.list) == 0

In [ ]:
import random
from copy import deepcopy

class Graph():
    
    def __init__(self, graph):
        self.graph = graph
        
    def list_of_nodes(self):
        return list(self.graph.keys())

    def choose_random_node(self):
        return random.choice(self.list_of_nodes())
        
    def choose_node_not_explored(self, graph, explored):
        nodes = self.list_of_nodes()
        nodes_copy = deepcopy(nodes)
        for node in explored:
            try:
                nodes_copy.remove(node) # Need try-remove statement
            except:
                pass
        if len(nodes_copy) == 0:
            return None
        return random.choice(nodes_copy)
        
    def bfs(self, start):
        '''
        Idea:
        Start at a node:
        Add it to a queue of nodes to be explored
        Look at all of its connections
        If they haven't been explored:
        1. Add them to an "explored" list
        2. Add them to the "to explore" queue
        Keep going until there is nothing left to explore!
        '''
        explored = []
        to_explore = Queue()
        
        to_explore.push(start)
        explored.append(start)
        
        while not to_explore.is_empty():
            el = to_explore.pop()
            new_nodes = self.graph.get(el, None)
            for node in new_nodes:
                if node not in explored:
                    explored.append(node)
                    to_explore.push(node)
        
        return explored
    
    def dfs(self, start, explored=Stack()):
        '''
        Idea:
        Start at a node:
        Add all of its connections to a LIFO stack
        '''
        explored.push(start)
        new_nodes = self.graph.get(start, None)
        for node in new_nodes:
            if node not in explored.list:
                self.dfs(node, explored)
            

        return explored.list

In [ ]:
graph = {}
graph['s'] = ['a', 'b']
graph['a'] = ['s', 'c']
graph['b'] = ['s', 'c', 'd']
graph['c'] = ['a', 'b', 'd', 'e']
graph['d'] = ['b', 'c', 'e']
graph['e'] = ['c', 'd']

g = Graph(graph)
print(g.dfs('a'))

# Solution

https://github.com/sestus/algorithms-stanford/blob/master/part_1/assignment4_strongly_connected_components/app/scc_finder.py

In [ ]:
class SccFinder(object):
    def __init__(self, input_file):
        self.scc_list = []
        with open(input_file) as file:
            self.finish_order = []
            self._graph = {}
            for line in file:
                (from_v, to_v) = tuple(number for number in line.split())
                self._add_edge_to_graph(int(from_v), int(to_v))

    def _add_edge_to_graph(self, from_v, to_v):
        if from_v in self._graph:
            self._graph[from_v].append(to_v)
        else:
            self._graph[from_v] = [to_v]
        if to_v in self._graph:
            self._graph[to_v].append(-from_v)
        else:
            self._graph[to_v] = [-from_v]

    def compute_finish_times(self):
        visited_nodes, finished_nodes = set(), set()
        for vertex in self._graph.keys():
            if vertex in visited_nodes:
                continue
            nodes_stack = [vertex]
            while nodes_stack:
                node = nodes_stack.pop()
                if node not in visited_nodes:
                    visited_nodes.add(node)
                    nodes_stack.append(node)
                    neighbors = (-edge for edge in self._graph[node] if edge < 0)
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            nodes_stack.append(neighbor)
                else:
                    if node not in finished_nodes:
                        self.finish_order.append(node)
                        finished_nodes.add(node)

    def compute_sccs(self):
        visited_nodes = set()
        assert (len(self.finish_order) == len(self._graph))
        for i in reversed(self.finish_order):
            if i in visited_nodes:
                continue
            nodes_stack = [i]
            size = 0
            while nodes_stack:
                node = nodes_stack.pop()
                if node not in visited_nodes:
                    size += 1
                    visited_nodes.add(node)
                    nodes_stack.append(node)
                    neighbors = (edge for edge in self._graph[node] if edge > 0)
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            nodes_stack.append(neighbor)
            self.scc_list.append(size)
        self.scc_list.sort(reverse=True)
        print(self.scc_list[:5])

# scc_finder = SccFinder("SCC.txt")
# scc_finder.compute_finish_times()
# scc_finder.compute_sccs()
# expected_sccs = [434821, 968, 459, 313, 211]
# print(scc_finder.scc_list[:5])

scc_finder = SccFinder("SCC_2.txt")
scc_finder.forward_graph
# scc_finder.compute_finish_times()
# scc_finder.compute_sccs()
# expected_sccs = [434821, 968, 459, 313, 211]
# print(scc_finder.scc_list[:5])
# scc_finder.finish_order




In [4]:
class SccFinder(object):
    def __init__(self, input_file):
        self.scc_list = []
        with open(input_file) as file:
            self.finish_order = []
            self.forward_graph = {}
            self.reverse_graph = {}
            for line in file:
                from_node, to_node = tuple(number for number in line.split())
                self._add_edges(int(from_node), int(to_node))

    def _add_edges(self, from_v, to_v):
        if from_v in self.forward_graph:
            self.forward_graph[from_v].append(to_v)
        else:
            self.forward_graph[from_v] = [to_v]
        if to_v in self.reverse_graph:
            self.reverse_graph[to_v].append(from_v)
        else:
            self.reverse_graph[to_v] = [from_v]

    def compute_finish_times(self):
        visited_nodes, finished_nodes = set(), set()
        for node in self.forward_graph.keys():
            if node in visited_nodes:
                continue
            to_explore = Stack()
            to_explore.push(node)
            while not to_explore.is_empty():
                next_node = to_explore.pop()
                if next_node not in visited_nodes:
                    visited_nodes.add(next_node)
                    to_explore.push(next_node)
                    neighbors = self.reverse_graph.get(next_node, [])
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            to_explore.push(neighbor)
                else:
                    if next_node not in finished_nodes:
                        self.finish_order.append(next_node)
                        finished_nodes.add(next_node)

    def compute_sccs(self):
        visited_nodes = set()
        assert (len(self.finish_order) == len(self.forward_graph))
        for i in reversed(self.finish_order):
            if i in visited_nodes:
                continue
            nodes_stack = Stack()
            nodes_stack.push(i)
            size = 0
            while not nodes_stack.is_empty():
                next_node = nodes_stack.pop()
                if next_node not in visited_nodes:
                    size += 1
                    visited_nodes.add(next_node)
                    nodes_stack.push(next_node)
                    neighbors = self.forward_graph.get(next_node, [])
                    for neighbor in neighbors:
                        if neighbor not in visited_nodes:
                            nodes_stack.push(neighbor)
            self.scc_list.append(size)
        self.scc_list.sort(reverse=True)
        print(self.scc_list[:5])

scc_finder = SccFinder("SCC.txt")
# scc_finder.forward_graph.keys()
scc_finder.compute_finish_times()
scc_finder.compute_sccs()
# expected_sccs = [434821, 968, 459, 313, 211]
# print(scc_finder.scc_list[:5])

# print(scc_finder.forward_graph)
# scc_finder.finish_order



[503872, 979, 459, 314, 219]


In [ ]:
list(range(9, 0, -1))

In [ ]:
def _add_edge_to_graph(_graph, from_v, to_v):
    if from_v in _graph:
        _graph[from_v].append(to_v)
    else:
        _graph[from_v] = [to_v]
    if to_v in _graph:
        _graph[to_v].append(-from_v)
    else:
        _graph[to_v] = [-from_v]
        
    return _graph
        
graph = {}
_add_edge_to_graph(graph, 1, 1)
_add_edge_to_graph(graph, 1, 2)
_add_edge_to_graph(graph, 1, 3)
_add_edge_to_graph(graph, 5, 1)

[-edge for edge in graph[1] if edge < 0]

In [ ]:
finish_order = []

def compute_finish_times(self):
    visited_nodes, finished_nodes = set(), set()
    for node in graph.keys():
        if node in visited_nodes:
            continue
        # Depth first search on graph
        # Go through nodes starting from vertex
        to_explore = Stack()
        to_explore.push(node)
        while not to_explore.is_empty():
            next_node = nodes_stack.pop()
            if next_node not in visited_nodes:
                visited_nodes.add(next_node)
                # nodes_stack keeps track of order of nodes visited
                to_explore.push(next_node)
                # neighbors lists all the nodes pointing at a node
                # Could do this with a reversed graph!
                neighbors = self.reverse_graph[next_node]
                # For each node pointing at a node
                for neighbor in neighbors:
                    # If we have not visited it
                    if neighbor not in visited_nodes:
                        # Add it to the nodes_stack, meaning it will get visited
                        to_explore.push(neighbor)
            else: # If the node *has* been visited
                if node not in finished_nodes: # And we have not added the node to "finished"
                    # Add it
                    finish_order.append(node)
                    finished_nodes.add(node)

This method below does it:

1. It creates a reverse graph from the original graph, which we don't want.
2. It doesn't return the SCCs in quite the right way.

In [ ]:
from collections import defaultdict
  
#This class represents a directed graph using adjacency list representation
class Min_Cut:
  
    def __init__(self):
        self.forward_graph = defaultdict(list) # default dictionary to store graph
        self.reverse_graph = defaultdict(list)
        self.stack = Stack()

    
    def create_graph_from_file(self, filename):
        with open(filename) as file:
            for line in file:
                from_node, to_node = tuple(number for number in line.split())
                self.add_edges(int(from_node), int(to_node))
            

    # function to add an edge to graph
    def add_edges(self, u, v):
        self.forward_graph[u].append(v)
        self.reverse_graph[u].append(v)

        
    # A function used by DFS
    def DFSUtil(self, v, visited=[]):
        # Mark the current node as visited and print it
        visited.append(v)
        # Recur for all the vertices adjacent to this vertex
        for i in self.forward_graph[v]:
            if i not in visited:
                self.DFSUtil(i, visited)
 
 
    def fillOrder(self, v, visited=[], stack=Stack()):
        # Mark the current node as visited 
        visited.append(v)
        # Recur for all the vertices adjacent to this vertex
        for i in self.forward_graph[v]:
            if i not in visited:
                self.fillOrder(i, visited, self.stack)
        self.stack.push(v)
     
 
    # Function that returns reverse (or transpose) of this graph
#     def getTranspose(self):
#         g = Graph(self.V)

#         # Recur for all the vertices adjacent to this vertex
#         for i in self.graph:
#             for j in self.graph[i]:
#                 g.addEdge(j,i)
#         return g
 
  
  
    # The main function that finds and prints all strongly
    # connected components
    def printSCCs(self):
        
        # Mark all the vertices as not visited (For first DFS)
        visited = []
        # Fill vertices in stack according to their finishing
        # times
        for i in range(len(self.forward_graph.keys())):
            if i not in visited:
                self.fillOrder(i, visited, self.stack)
 
        # Create a reversed graph
#         gr = self.getTranspose()
         
        # Mark all the vertices as not visited (For second DFS)
        visited_reverse = []
 
        # Now process all vertices in order defined by Stack
        sccs = []
        while not self.stack.is_empty():
            i = self.stack.pop()
            if i not in visited_reverse:
                out = reverse_graph.DFSUtil(i, visited_reverse)
                sccs.append(len(out))

In [ ]:
from collections import defaultdict
  
#This class represents a directed graph using adjacency list representation
class Graph:
  
    def __init__(self,vertices):
        self.V= vertices #No. of vertices
        self.graph = defaultdict(list) # default dictionary to store graph
  
    # function to add an edge to graph
    def addEdge(self,u,v):
        self.graph[u].append(v)
  
    # A function used by DFS
    def DFSUtil(self,v,visited):
        # Mark the current node as visited and print it
        visited[v]= True
        print(v,)
        #Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if visited[i]==False:
                self.DFSUtil(i,visited)
 
 
    def fillOrder(self,v,visited, stack):
        # Mark the current node as visited 
        visited[v]= True
        #Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if visited[i]==False:
                self.fillOrder(i, visited, stack)
        stack = stack.append(v)
     
 
    # Function that returns reverse (or transpose) of this graph
    def getTranspose(self):
        g = Graph(self.V)
 
        # Recur for all the vertices adjacent to this vertex
        for i in self.graph:
            for j in self.graph[i]:
                g.addEdge(j,i)
        return g
 
  
  
    # The main function that finds and prints all strongly
    # connected components
    def printSCCs(self):
         
        stack = []
        # Mark all the vertices as not visited (For first DFS)
        visited =[False]*(self.V)
        # Fill vertices in stack according to their finishing
        # times
        for i in range(self.V):
            if visited[i]==False:
                self.fillOrder(i, visited, stack)
 
        # Create a reversed graph
        gr = self.getTranspose()
         
        # Mark all the vertices as not visited (For second DFS)
        visited =[False]*(self.V)
 
        # Now process all vertices in order defined by Stack
        while stack:
            i = stack.pop()
            if visited[i]==False:
                gr.DFSUtil(i, visited)
                print("")

In [ ]:
g = Graph(5)
g.addEdge(1, 0)
g.addEdge(0, 2)
g.addEdge(2, 1)
g.addEdge(0, 3)
g.addEdge(3, 4)
print ("Following are strongly connected components " +
                           "in given graph")
g.printSCCs()

In [ ]:
g = Min_Cut()

In [ ]:
g.create_graph_from_file("SCC.txt")

In [ ]:
g.printSCCs()

In [ ]:
len(g.stack.list)